# Stock Price Visualization with Nixtla SDK

This notebook demonstrates how to use the Nixtla SDK to forecast stock prices, detect anomalies, and perform cross-validation. 

**Requirements:**
- `nixtla`
- `yfinance`
- `matplotlib`
- A valid Nixtla API Key (Get yours at [dashboard.nixtla.io](https://dashboard.nixtla.io))

In [ ]:
import os
import pandas as pd
import yfinance as yf
from nixtla import NixtlaClient
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Data Acquisition
We fetch historical stock data using `yfinance`. For this example, we'll use Apple Inc. (AAPL).

In [ ]:
def fetch_stock_data(ticker: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Fetches stock data from yfinance and preprocesses it for Nixtla.
    """
    print(f"Fetching data for {ticker}...")
    try:
        # Fetch data
        df = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
        if df.empty:
            raise ValueError("No data fetched.")
        
        # Reset index to get 'Date' as a column
        df = df.reset_index()
        
        # Flatten MultiIndex columns if present
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

        # Select and rename columns
        date_col = [c for c in df.columns if 'Date' in str(c)][0]
        target_col = 'Close' 
        
        df = df[[date_col, target_col]].copy()
        df.columns = ['ds', 'y']
        df['unique_id'] = ticker
        
        # Ensure correct types and handle timezone
        df['ds'] = pd.to_datetime(df['ds']).dt.tz_localize(None)
        
        # Resample to fill missing dates (weekends/holidays) to ensure regular frequency
        # This is crucial for TimeGPT if not using 'B' frequency which can be strict about holidays
        print("Resampling data to Daily frequency (forward filling)...")
        df = df.set_index('ds').asfreq('D').ffill().reset_index()
        df['unique_id'] = ticker # unique_id might become NaN after resampling
        
        print(f"Data fetched and resampled successfully. Shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

In [ ]:
# Parameters
TICKER = "AAPL"
START = "2023-01-01"
END = "2024-01-01"

df = fetch_stock_data(TICKER, START, END)
df.head()

## 2. Initialize Nixtla Client
Ensure you have your API key set in your environment variables or pass it directly.

In [ ]:
api_key = os.environ.get("NIXTLA_API_KEY")
# api_key = 'YOUR_API_KEY_HERE' # Uncomment and set if not in env

if not api_key:
    print("WARNING: NIXTLA_API_KEY not found.")
else:
    client = NixtlaClient(api_key=api_key)

## 3. Forecasting
We forecast the next 14 days with 80% and 90% prediction intervals.

In [ ]:
if api_key:
    try:
        forecast_horizon = 14
        fcst_df = client.forecast(df, h=forecast_horizon, level=[80, 90], freq='D')
        print("Forecast generated successfully.")
        
        client.plot(df, fcst_df, level=[80, 90])
    except Exception as e:
        print(f"Forecast failed: {e}")

## 4. Anomaly Detection
Identify anomalies in the historical data.

In [ ]:
if api_key:
    try:
        anomalies_df = client.detect_anomalies(df, time_col='ds', target_col='y', level=99, freq='D')
        print("Anomaly detection complete.")
        
        client.plot(df, anomalies_df, time_col='ds', target_col='y', plot_anomalies=True)
    except Exception as e:
        print(f"Anomaly detection failed: {e}")

## 5. Cross Validation
Perform backtesting to evaluate model performance.

In [ ]:
if api_key:
    try:
        cv_df = client.cross_validation(
            df, 
            h=7, 
            n_windows=3, 
            step_size=7,
            level=[80, 90],
            freq='D'
        )
        print("Cross validation complete.")
        
        client.plot(df, cv_df, level=[80, 90])
    except Exception as e:
        print(f"Cross validation failed: {e}")